In [200]:
# use qpcr conda environment
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
# import matplotlib.patches as mpatches
from matplotlib import rcParams
import seaborn as sns
from statannotations.Annotator import Annotator
import os

In [ ]:
#need tp split No TF data by date

In [201]:
#function to read in data
def read_data(file):
    data = pd.read_csv(file, sep='\t', header=0)
    return data

In [202]:
#make plot
def make_plot(prom,TF,df,order, pair, fontsize,ax,folder,plot_title=None):
    """function to make and save plots"""
    # plt.figure(figsize=(fig_width,fig_height))
    
    #order_new = pair[0]
    fig_args = {'x':'TF_added', 'y':'corrected_luminescence','data':df, 'order':order, 'dodge':True,}#'ax':ax
    configuration = {'test':'t-test_welch', 'text_format':'star', 'pvalue_thresholds':[[1e-3, "***"],[1e-2, "**"],[0.05, "*"],[1, "ns"]]}#"pairs":list(box_pairs_significant.keys()),"pvalues":list(box_pairs_significant.values()), 'loc':'inside'
    sns.barplot(**fig_args, linewidth=2,  errcolor="black", edgecolor="black", ci=68, errwidth=1,capsize=0.4,color='#BFBFBF',ax=ax)#facecolor=(1,1,1,1),
    #fig.set_aspect(0.5)
    sns.swarmplot(**fig_args,color='black',ax=ax, size=3)
    #add stats
    annotator = Annotator(ax, pair, **fig_args,verbose=False, show_non_significant=False)
    annotator.configure(**configuration)#loc='inside'
    ax, test_results = annotator.apply_and_annotate()
    with open(f'../../../../data/plots/luminescence/{folder}/individual/stats.txt', 'a') as f:                            
        for res in test_results:
            f.write(f'{str(prom)},{pair},{str(res.data)}\n')
    # change axes labels
    ax.set_ylabel('Normalised expression (a.u.)')
    ax.set_xlabel('TF added')
    # ax.set_title(plot_title)
    #add title
    #plt.title(prom[:len(prom)-20], y=1,size=fontsize)
    if plot_title==None:
        pass
        # ax.set_title(f'Co-expression\n 35s:{TF[1:]} + {prom}p:LucN', y=1,size=fontsize)
    if plot_title!=None:
        ax.set_title(plot_title, y=1,size=fontsize)
    
    #make subplot xticks diagonal 

    # ax.set_xticklabels(, rotation=90, ha="right",rotation_mode="anchor")
    # plt.xticks(rotation=90, ha='center')#ha='right'
    #plt.xlabel(x_label)
    # tight layout
    # plt.tight_layout()
    # save figure
    
    #don't show plot in notebook
    # plt.cla()  # clear axis              
    # plt.close('all')    

In [203]:
#function to make subplots and iterate over promoters
def make_subplots(df,fontsize,TFs_of_interest,folder):
    """function to make subplots"""
    # get prom_names as list
    
    proms = list(df['Promoter'].unique())

    #divide number of subplots by 4 to get number of rows
    
    #iterate over promoters
    for prom in proms:
        # print(f'prom={prom}')
        #  for sample in samples:
        #         if sample != 'Col-0':
        #             plateID_value = temp_df.loc[temp_df['Sample'] == sample, 'plateID'].values[0]
        #         # print(plateID_value)
        #             if plateID_value == '22.08.22_plate1':   
        #                 #print(f'platevalueis{plateID_value}')                
        #                 df_new = temp_df[(temp_df.Sample == sample) | ((temp_df.Sample == 'Col-0') & (temp_df.plateID == plateID_value))]

        #             if plateID_value == '22.08.22_plate2':

        #                 df_new = temp_df[(temp_df.Sample == sample) | ((temp_df.Sample == 'Col-0') & (temp_df.plateID == plateID_value))]
        
        #get list of TFs for each promoter
        TFs_orig = df[df['Promoter']==prom]['TF_added'].unique()
        #sort TFs alphabetically
        #TFs_orig = sorted(TFs_orig)
        #assign list of TFs of interest with this well calibrator
        # print(f'TFs_of_interest={TFs_of_interest}')
        #sort based on custom order
        TFs = []
        for i in range(len(TFs_of_interest)):
            if TFs_of_interest[i] in TFs_orig:
                # print(f'TFs_of_interest[i]={TFs_of_interest[i]}')
                TFs+=[TFs_of_interest[i]]
        TFs = sorted(TFs)
        # print(f'TFs={TFs}')
        # print(f'TFs_orig={TFs_orig}')
        
        #define number of subplots
        number_of_subplots = len(TFs)
        #make subplots for each promoter
        number_of_rows = math.ceil(number_of_subplots/4) 
        height = 4.8*number_of_rows
        width = 10
        #make subplots equal to number_of_subplots
        fig, axes = plt.subplots(nrows=number_of_rows, ncols=4, figsize=(width, height), sharex=False)
        #flatten axis array
        axes = axes.flatten()
        axes_count = 0
        #make promoter df
        promoter_df = df[df['Promoter']==prom]
        #iterate over TFs
        for TF in TFs:
            if TF != 'No TF':
                # print(f'TF={TF}')
                # print(f'axes_count={axes_count}')
                
                #get data for each TF
                
                TF_df = promoter_df[(promoter_df['TF_added']==TF)|(promoter_df['TF_added']=='No TF')]
                #define pair and order
                pair = [('No TF', TF)]
                order= pair[0]
                
                # order = ['No TF',TF]
                # pair = [TF,'No TF']
                #make plot
                make_plot(prom,TF,TF_df,order, pair, fontsize,axes[axes_count],folder)
                axes_count+=1
        #at the end make plot xticks 90 edgrees
        plt.xticks(rotation=90, ha='center')
        plt.savefig(
            f'../../../../data/plots/luminescence/{folder}/individual/{prom}_barplot.pdf',
            format="pdf",
            bbox_inches="tight",transparent=True)
        plt.savefig(
            f'../../../../data/plots/luminescence/{folder}/individual/{prom}_barplot.svg',
            format="svg",
            bbox_inches="tight",transparent=True)
        #remove plot
        plt.cla()  # clear axis
        plt.close('all')
        #make plot
        # make_plot(prom,TF,df,order, pair, fig_width, fig_height,bar_width, fontsize,axes[axes_count],plot_title)
        #increase axes count
        # axes_count+=1


    

In [204]:
#set matplotlib rc parameters
def set_rc_params():
    #set matplotlib default parameters
    rcParams['xtick.major.width'] = 2
    rcParams['ytick.major.width'] = 2
    rcParams['axes.linewidth'] = 2
    rcParams['lines.linewidth'] = 2
    #remove top and right lines
    rcParams['axes.spines.top'] = False
    rcParams['axes.spines.right'] = False
    #font size
    fontsize = 20
    rcParams['font.size'] = fontsize
    #for getting the microsoft font Arial working, please follow this guide: https://alexanderlabwhoi.github.io/post/2021-03-missingfont/
    rcParams['font.family'] = 'sans-serif'
    rcParams['font.sans-serif'] = ['Arial']
    #allow font to be edited later in pdf editor
    #make svg text editable
    rcParams['svg.fonttype'] = 'none'
    rcParams ['pdf.fonttype'] = 42 
    #align y-axis top most tick with end of axis
    rcParams['axes.autolimit_mode'] = 'round_numbers'
    #set margins to ensure any error bars fit
    rcParams['axes.xmargin'] = 0.2
    rcParams['axes.ymargin'] = 0.2
    #define bar width
    #bar_width = 0.65
    #allow math text to be displayed
    #rcParams['mathtext.default'] = 'regular'
    return fontsize
    

In [205]:
#main function
def main():
    #data location
    data_dir = '../../../../data/plots/luminescence/all_combined/coexpression_data_native.tsv'
    #read in tsv
    df = read_data(data_dir)
    #delete the stats file 
    
    os.remove(f'../../../../data/plots/luminescence/all_combined/individual/stats.txt')

    

    #make new folder
    #create individual plot folder name
    #make directory for the plots to be exported to
    dirName = f'../../../../data/plots/luminescence/all_combined/individual'
    try:
        # Create target Directory
        os.mkdir(dirName)
        print("Directory " , dirName ,  " created") 
    except FileExistsError:
        print("Directory " , dirName ,  " already exists")
    

    
    
    #make plots
    
    fontsize = set_rc_params()
    #filter out if condition is not nitrogen
    df_filtered = df[df['condition']=='nitrogen']
    filter_tf_list = ['No TF','ARF9','ARF18','ARF18+ARF9','ARF9_1μM_NAA','ARF18_1μM_NAA','ARF18+ARF9_1μM_NAA']#'ARF18+IAA33_1μM_NAA','ARF18+IAA33'
    TFs_of_interest = ['No TF','ANAC032','ANR1','bZIP3','DREB26','HHO2','NLP6','NLP7','TGA1']
    #filter out if TF_Added is item in filter_tf_list


    #df_filtered = df_filtered[~df_filtered['TF_added'].isin(filter_tf_list)]
    make_subplots(df_filtered, fontsize, TFs_of_interest,'all_combined')



In [206]:
#main function call
if __name__ == '__main__':
    main()

Directory  ../../../../data/plots/luminescence/all_combined/individual  already exists
